In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
# Feature Selection (With binary data)
from sklearn.feature_selection import mutual_info_classif

def feature_selection(inputfile, compared_with, compared_for, threshold):
    data = pd.read_CSV(inputfile)
    filtered_features = []
    features = data[compared_with]
    class_label = data[compared_for]
    mi_arr = mutual_info_classif(X=features, y=class_label, discrete_features=True)

    for feature, mi in zip(features.columns, mi_arr):
        print(f'MI value for feature "{feature}": {mi:.4f}')
        
        if(mi >= threshold): 
            filtered_features.append(feature)
            
    return ('\nFeature set after filtering with MI:', filtered_features)